In [2]:
# import filtering  
# df = filtering.moses_to_df("data/en-si/Ubuntu.en-si.en", "data/en-si/Ubuntu.en-si.si", "english", "sinhala")
# e1 = filtering.to_multilingual_embedding("english", df["english"], "labse")
# e2 = filtering.to_multilingual_embedding("sinhala", df["sinhala"], "labse")
# ss = filtering.find_similarity_score(e1, e2)

In [3]:
import bucc_style_dataset as bsd

In [4]:
si_mono_lines = []
eng_mono_lines = []
with open("data/en-si/sin_wikipedia_2021_30K-sentences.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.removesuffix("\n")
        split  =line.split("\t")
        si_mono_lines.append(split[1])
with open("data/en-si/eng_news_2024_30K-sentences.txt", "r", encoding="utf-8") as f:
    for line in f: 
        line = line.removesuffix("\n")
        split  =line.split("\t")
        eng_mono_lines.append(split[1])


In [5]:
si_par_lines = []
eng_par_lines = []
with open("data/en-si/eng_Latn.dev", "r", encoding="utf-8") as f:
    for line in f:
        line = line.removesuffix("\n")
        eng_par_lines.append(line)
with open("data/en-si/sin_Sinh.dev", "r", encoding="utf-8") as f:
    for line in f: 
        line = line.removesuffix("\n")
        si_par_lines.append(line)

In [6]:
train_list, test_list = bsd.split_shuffle_create_corpus(eng_mono_lines, si_mono_lines, eng_par_lines, si_par_lines)

Dataset length: 30000.
7500 22500
Dataset length: 30000.
7500 22500
Dataset length: 997.
249 748
Dataset length: 997.
249 748
False False False False
7500 monolingual source sentences.
7500 monolingual target sentences.
Whole corpus:
7749 monolingual source sentences.
7749 monolingual target sentences.
src-0000000	A staggering $1.96 million in ENA tokens were awarded to the highest airdrop recipient, demonstrating the significant impact of this distribution. trg-0000000	ඇසිපිල්ලම් තුනීසිවියකින් (ආරක්ෂාකාරී තුන්වන ඇසිපිය) යුක්ත ඇස් විශාල වටකුරු ස්වභාවයක් ගනීයි.
22500 monolingual source sentences.
22500 monolingual target sentences.
Whole corpus:
23248 monolingual source sentences.
23248 monolingual target sentences.
src-0000000	I have written to Sky New registering my disappointment with them, suggest others do the same. trg-0000000	ප්‍රවෘති ඉදිරිපත් කරන්නන්ගේ බොහෝ පුවත් “ පාර්සල් ස්වරූපයක් ගනී’.


In [17]:
en_list = train_list[0].split("\n")
si_list = train_list[1].split("\n")
en_corpus_lines = []
si_corpus_lines = []
for line in en_list:
    line = line.split("\t")[1]
    en_corpus_lines.append(line)
for line in si_list:
    line = line.split("\t")[1]
    si_corpus_lines.append(line)
import filtering
e1 = filtering.to_multilingual_embedding("english", en_corpus_lines, "labse")
e2 = filtering.to_multilingual_embedding("sinhala", si_corpus_lines, "labse")    

In [18]:
source_lines = [f"{e1.shape[0]} {e1.shape[1]}"]
target_lines = [f"{e2.shape[0]} {e2.shape[1]}"]
for sent, encoding in zip(en_corpus_lines, e1):
    sent = sent.replace(" ", "_")
    encoding_str = " ".join([f"{x:.4f}" for x in encoding])
    source_lines.append(f"{sent} {encoding_str}")
for sent, encoding in zip(si_corpus_lines, e2):
    sent = sent.replace(" ", "_")
    encoding_str = " ".join([f"{x:.4f}" for x in encoding])
    target_lines.append(f"{sent} {encoding_str}")

In [24]:
with open("source.vec", "w", encoding="utf-8") as f: 
    for line in source_lines:
        f.write(f"{line}\n")
with open("target.vec", "w", encoding="utf-8") as f: 
    for line in target_lines:
        f.write(f"{line}\n")


In [2]:
import bilingual_nearest_neighbor as bnn 
bnn.main(source_embeddings="evaluation/en.source.vec", target_embeddings="evaluation/si.target.vec", output="evaluation/en-si.output.txt", binary=0)

In [14]:
gold_file_dict = {}
with open("data/en-si/eng_Latn.dev", "r", encoding="utf-8") as f1, open("data/en-si/sin_Sinh.dev", "r", encoding="utf-8") as f2:
    for line1, line2 in zip(f1, f2):
        line1 = line1.removesuffix("\n")
        line2 = line2.removesuffix("\n")
        line1 = line1.replace("\u200d", "")
        line2 = line2.replace("\u200d", "")
        gold_file_dict[line1] = line2

In [25]:
with open("evaluation/en-si.output.txt", "r", encoding="utf-8") as f:
    test_dict = {}
    for line in f:
        split = line.split("\t")
        split = [item.replace("_", " ") for item in split]
        split = [item.replace("\u200d", "") for item in split]
        split = split[:2]
        if split[0] in gold_file_dict.keys():
            test_dict[split[0]] = split[1]

In [27]:
with open("evaluation/mined.en", "w", encoding="utf-8") as f1, open("evaluation/mined.si", "w", encoding="utf-8") as f2:
    for s1, s2 in test_dict.items():
        f1.write(s1 + "\n")
        f2.write(s2 + "\n")

In [29]:
import filtering 
filtering.main(files=["evaluation/mined.en", "evaluation/mined.si"], langs=["english", "sinhala"], output="evaluation/filtered.en-si.tsv", model="labse")

100%|██████████| 149/149 [00:00<00:00, 775373.82it/s]
2025-05-16 11:54:19,278 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: xlm-roberta-base
100%|██████████| 149/149 [00:17<00:00,  8.40it/s]
2025-05-16 11:54:38,875 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base
INFO:simalign.simalign:Initialized the EmbeddingLoader with model: xlm-roberta-base
100%|██████████| 149/149 [00:00<00:00, 3182.26it/s]


[['▁"', 'We'], ['▁were'], ['▁all'], ['▁simply'], ['▁in'], ['▁shock', ',', '"'], ['▁the'], ['▁mother'], ['▁state', 'd', '.']] [['▁"', '"', '“', 'අපි'], ['▁හැමෝම'], ['▁', 'කම්', 'පන', 'යට'], ['▁පත්ව', 'ුණා', ',', '”', '"'], ['▁මව'], ['▁පැ', 'ව', 'සු', 'වා'], ['▁ය', '."']] [(0, 0), (2, 1), (5, 3), (7, 4)]
[['▁In'], ['▁2010,'], ['▁while'], ['▁national', 'ised', ','], ['▁the'], ['▁current'], ['▁high'], ['▁street'], ['▁bank'], ['▁Northern'], ['▁Rock'], ['▁pl', 'c'], ['▁was'], ['▁split'], ['▁from'], ['▁the'], ['▁‘', 'bad'], ['▁bank', '’', ','], ['▁Northern'], ['▁Rock'], ['▁(', 'A', 'sset'], ['▁Management', ').']] [['▁2010'], ['▁දී', ','], ['▁ජන', 'ස', 'තුව'], ['▁පවතින'], ['▁විට', ','], ['▁', 'වත්මන්'], ['▁මහා'], ['▁වී', 'දි'], ['▁බැංකුව'], ['▁වන'], ['▁නොද', 'ර්', 'න්'], ['▁රො', 'ක්'], ['▁පී', '.', 'එ', 'ල්', '.', 'සී', ','], ['▁‘', 'අ', 'යෝ', 'ග්', 'ය'], ['▁බැංකුව', '’', ','], ['▁හෙවත්'], ['▁නොද', 'ර්', 'න්'], ['▁රො', 'ක්'], ['▁(', 'වත්', 'කම්'], ['▁කළමනාකරණ', 'ය', ')'], ['▁වෙත', 'ින්'], ['▁ව

100%|██████████| 149/149 [00:00<00:00, 366540.35it/s]

Raw corpus: 249

After dropping duplicates: 249

After removing length based outliers: 245

After performing language identification: 245

After filtering based on similarity scores: 149

After filtering based on word alignment: 142

0.18251046932458745
